<a href="https://colab.research.google.com/github/JuliCSD/fine-tuning-mobileBERT/blob/main/Mobile_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers --quiet
!pip install sentencepiece --quiet
!pip install datasets --quiet
!pip install transformers[torch] --quiet
!pip install accelerate -U --quiet
!pip install trainer --quiet
!pip install pytorch_lightning --quiet
!pip install evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.4 MB/s eta 0:00:00


In [ ]:
#code pour pipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="google/mobilebert-uncased",
    tokenizer="google/mobilebert-uncased"
)
for i in range(5):
  print(i+1, "- ",
      fill_mask(f"I want to {fill_mask.tokenizer.mask_token} ")[i]['token_str'], '\n'
  )

pytorch_model.bin:   0%|          | 0.00/147M [00:00<?, ?B/s]

1 -  . 

2 -  be 

3 -  see 

4 -  have 

5 -  go 



In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive/')
# Chemin dans gdrive
csv_file_path = '/content/gdrive/MyDrive/BD.csv'

In [ ]:
import pandas as pd
# Lire csv grace a Pandas
df = pd.read_csv(csv_file_path)
df

,Original Sentence,Lemmatized Sentence,File Name,Phrase mélangée (=1 si mélangée)
0,"Voici le bateau sur lequel nous étions, d'aill...",voici bateau lequel nous être ailleurs,ted,0
1,Il remonte encore la chaîne alimentaire.,il remonter encore chaîne alimentaire,ted,0
2,Et cela ne devrait pas nous surprendre.,celer ne devoir pas nous surprendre,ted,0
3,"D'où le titre ""Le printemps silencieux"".",où titre printemps silencieux,ted,0
4,N'ayez pas honte.,ne avoir pas hont,ted,0
...,...,...,...,...
123391,"Cet outil deviendra mon meilleur ami, et le vô...",outil devenir meilleur ami vôtre,ted,0
123392,C'est vraiment un immense défi.,ce être vraiment immense défi,ted,0
123393,La frontière est celle du monde réel.,frontière être celui monde réel,ted,0
123394,La communauté aveugle étudie cette frontière t...,communauté aveugle étudier frontière technique...,ted,0


In [ ]:
from sklearn.model_selection import train_test_split
from transformers import MobileBertTokenizer,  MobileBertForMaskedLM, AdamW

# Diviser le jeu de données en entraînement et validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Définir le tokenizer BERT
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')


In [ ]:
from datasets import Dataset
# set small dataset

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

train_ds_small = train_ds.select([i for i in range(100)])
val_ds_small = val_ds.select([i for i in range(100)])

train_df_small = train_ds_small.to_pandas()
val_df_small = val_ds_small.to_pandas()

In [ ]:
from torch.utils.data import Dataset

# Définir une classe de dataset personnalisée pour MLM
class CustomMLMDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=128):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = str(self.dataframe.iloc[idx, 0])
        inputs = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': inputs['input_ids'].flatten()  # Les étiquettes sont les mêmes que les entrées pour MLM
        }


In [ ]:
from torch.utils.data import DataLoader

# Créer les datasets pour MLM
train_mlm_dataset = CustomMLMDataset(train_df, tokenizer)
val_mlm_dataset = CustomMLMDataset(val_df, tokenizer)

# Créer les dataloaders pour MLM
train_mlm_dataloader = DataLoader(train_mlm_dataset, batch_size=8, shuffle=True)
val_mlm_dataloader = DataLoader(val_mlm_dataset, batch_size=8, shuffle=False)

# Charger le modèle BERT pour MLM
model = MobileBertForMaskedLM.from_pretrained('google/mobilebert-uncased')

# Définir l'optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
!pip install tensorboardX --quiet

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import torch
from torch.utils.data import Dataset
from tensorboardX import SummaryWriter
from tqdm import tqdm

# Entraîner le modèle pour MLM
num_epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

writer = SummaryWriter()

for epoch in range(num_epochs):
    model.train()

    # Utiliser tqdm pour afficher la barre de progression
    train_mlm_dataloader_tqdm = tqdm(train_mlm_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}', position=0, leave=True)

    train_loss = 0.0
    for batch in train_mlm_dataloader_tqdm:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss+= loss.item()

    train_loss /= len(train_mlm_dataloader_tqdm)
    writer.add_scalar('Loss/train', train_loss, epoch)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_mlm_dataloader:
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**inputs)
            val_loss += outputs.loss.item()

    val_loss /= len(val_mlm_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss}")
    writer.add_scalar('Loss/val', val_loss, epoch)

# Sauvegarder le modèle fine-tuné pour MLM
model.save_pretrained("repertoire")

# Sauvegarder le tokenizer
tokenizer.save_pretrained("repertoire")

writer.close()

Epoch 1/3:   2%|▏         | 288/12340 [29:55<19:31:36,  5.83s/it]

In [ ]:
%tensorboard --logdir runs